In [203]:
import pandas as pd
import numpy as np 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [204]:
#Import the needed datas
#Read the datas
movies = pd.read_csv('titles.csv')
act_on_movies = pd.read_pickle('act_on_movies.pkl')

In [205]:
title_full = movies.merge(act_on_movies, how = 'left', on = 'tconst')
title_full.drop(columns = ['endYear'], inplace=True)
title_full_noNaN = title_full.dropna()
title_full_noNaN.reset_index(inplace = True)

In [206]:
title_full_noNaN['genres']

0                            Romance
1                                 \N
2         Action,Adventure,Biography
3                              Drama
4                              Drama
                     ...            
294898                 Drama,History
294899         Adventure,History,War
294900                        Comedy
294901                         Drama
294902                            \N
Name: genres, Length: 294903, dtype: object

In [207]:
#Get a list of genres per movie
title_full_noNaN= title_full_noNaN[title_full_noNaN['genres'] != '\\N']
title_full_noNaN['genres'] = title_full_noNaN['genres'].apply(lambda x : x.split(','))

#title_full_noNaN['genres'] = title_full_noNaN['genres'].apply(lambda x : "[]" if x == '\\N' else x)\
 #   .apply(lambda x : x.split(','))

In [208]:
#Get the median
med_tvMovie = title_full_noNaN['runtimeMinutes'][(title_full_noNaN['titleType']=='tvMovie') & (title_full_noNaN['runtimeMinutes']!='\\N')].median()
med_movie = title_full_noNaN['runtimeMinutes'][(title_full_noNaN['titleType']=='movie') & (title_full_noNaN['runtimeMinutes']!='\\N')].median()

#Replace the missing values
title_full_noNaN['runtimeMinutes'][(title_full_noNaN['titleType']=='movie') & (title_full_noNaN['runtimeMinutes']=='\\N')]=med_movie
title_full_noNaN['runtimeMinutes'][(title_full_noNaN['titleType']=='tvMovie') & (title_full_noNaN['runtimeMinutes']=='\\N')]=med_tvMovie

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_21948\2582152373.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title_full_noNaN['runtimeMinutes'][(title_full_noNaN['titleType']=='movie') & (title_full_noNaN['runtimeMinutes']=='\\N')]=med_movie
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_21948\2582152373.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title_full_noNaN['runtimeMinutes'][(title_full_noNaN['titleType']=='tvMovie') & (title_full_noNaN['runtimeMinutes']=='\\N')]=med_tvMovie


In [213]:
df.drop_duplicates(subset = df[['originalTitle' ,'primaryTitle', 'startYear']], inplace = True)

In [214]:
df = df[df['titleType']=='movie']
df['runtimeMinutes'] = df['runtimeMinutes'].apply(lambda x : int(x))
df = df[df['runtimeMinutes']>60]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_21948\3234267398.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['runtimeMinutes'] = df['runtimeMinutes'].apply(lambda x : int(x))


In [223]:
s = df['genres']
df = pd.concat([df , pd.get_dummies(s.explode()).groupby(level=0).sum()], 
          axis = 1)

,index,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,...,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western
2,3,3,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"[Action, Adventure, Biography]",...,0,0,0,0,0,0,0,0,0,0
3,4,4,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,90,[Drama],...,0,0,0,0,0,0,0,0,0,0
4,5,5,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,92,[Drama],...,0,0,0,0,0,0,0,0,0,0
5,6,6,tt0000630,movie,Hamlet,Amleto,0,1908,92,[Drama],...,0,0,0,0,0,0,0,0,0,0
6,8,8,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,120,"[Adventure, Fantasy]",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294895,785139,785139,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"[Action, Adventure, Thriller]",...,0,0,0,0,0,0,0,1,0,0
294897,785142,785142,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,84,[Thriller],...,0,0,0,0,0,0,0,1,0,0
294898,785143,785143,tt9916362,movie,Coven,Akelarre,0,2020,92,"[Drama, History]",...,0,0,0,0,0,0,0,0,0,0
294899,785144,785144,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019,92,"[Adventure, History, War]",...,0,0,0,0,0,0,0,0,1,0


In [237]:
#isAdult, averageRating, runtimeMinutes, (Year?), genres (get_dummies)
columns_of_interest = ['isAdult', 'runtimeMinutes', 'averageRating'] + list(df.iloc[:, -26:].columns)
X= df[columns_of_interest]
distanceKNN = NearestNeighbors(n_neighbors = 6).fit(X)
title = df['originalTitle'][294897]
coord = distanceKNN.kneighbors(df.loc[df['originalTitle']==title, columns_of_interest])

In [238]:
for i in range(1, 6):
    print('You entered :', title, ', so we recommand :', 'TOP', i, ':' , df['originalTitle'].iloc[coord[1][0][i]])

You entered : Il talento del calabrone , so we recommand : TOP 1 : Belinda dos Orixás na Praia dos Desejos
You entered : Il talento del calabrone , so we recommand : TOP 2 : Unspeakable
You entered : Il talento del calabrone , so we recommand : TOP 3 : La furia de Mackenzie
You entered : Il talento del calabrone , so we recommand : TOP 4 : In Utero
You entered : Il talento del calabrone , so we recommand : TOP 5 : The Name of the Game Is Kill!


In [ ]:
# Problems : 1 - Ratings in recommandation system, 
# 2 - Streamlit search bar very slow to load all these things, 
# 3 - What weight to give to every columns in the algorithm (most important : Genres ?)